In [1]:
def fnv(s):
    hval = 0x811c9dc5
    fnv_32_prime = 0x01000193
    uint32_max = 2 ** 32
    for c in s:
        hval = hval ^ ord(c)
        hval = (hval * fnv_32_prime) % uint32_max
    # Convert fnv1a-32 to 24
    return ((hval>>24) ^ hval) & 0xFFFFFF


In [0]:
def apply_to_lines(file, fn, state=None):
    with open(file) as f:
        line_num = 1
        for line in f.readlines():
            line = line[:-1]
            if not fn(line_num, line, state):
                return False
    return True


In [0]:

def add_hash(line_num, package, hashes):
    h = fnv(package)
    hashes.append(h)
    return True

hashes = []
apply_to_lines("/Users/grwheele/Downloads/train.txt", add_hash, hashes)
hashes = sorted(hashes)

print(len(hashes))


In [0]:
offsets = []
index = {}
last_h = 0
index[0] = 0

for i, h in enumerate(hashes):
    diff = h - last_h
    last_h = h

    if diff:
        # we use 0 for a continuation
        while diff > 255:
            offsets.append(0)
            diff -= 255
        offsets.append(diff)
    if i and i % 1000 == 0:
        index[h] = len(offsets)
    
print(len(offsets))


In [0]:
def is_in_filter(package):
    h = fnv(package)
    # we could speed this up with a binary search but
    # it's not worth it yet
    last_k = 0
    for k in index.keys():
        if k > h:
            break
        last_k = k
    diff = h - last_k
    offset = index[last_k]
    while diff >= 0:
        if diff == 0:
            return True
        while offsets[offset] == 0:
            diff -= 255
            offset += 1
        diff -= offsets[offset]
        offset += 1
    return False


In [0]:
def assert_found(line_num, line, _):
    if is_in_filter(line):
        return True
    print(f"Failed at {line_num}: {line}")
    return False


In [0]:
def count_found(line_num, line, state):
    if is_in_filter(line):
        state[0] += 1
    else:
        state[1] += 1
    return True


In [0]:
    
if apply_to_lines("/Users/grwheele/Downloads/train.txt",assert_found):
    print("All found!")
        

In [0]:
counts = [0, 0]
apply_to_lines("/Users/grwheele/Downloads/test.txt", count_found, counts)
print(f"{counts[0]} false positives, {counts[1]} true positives")


